In [1]:
import pandas as pd

In [2]:
% ls data

mt_bill_actions.csv           mt_bills.csv
mt_bill_legislator_votes.csv  mt_committees.csv
mt_bill_sponsors.csv          mt_legislator_roles.csv
mt_bill_votes.csv             mt_legislators.csv


In [3]:
bills = pd.read_csv('data/mt_bills.csv')
actions = pd.read_csv('data/mt_bill_actions.csv')
sponsors = pd.read_csv('data/mt_bill_sponsors.csv')
legislators = pd.read_csv('data/mt_legislators.csv', dtype={'district': str})

In [4]:
actions_17 = actions.loc[actions['session'] == 2017]
sponsors_17 = sponsors.loc[sponsors['session'] == 2017]

In [5]:
bills['type'].value_counts(dropna=False)

bill                4224
joint resolution     277
resolution           251
Name: type, dtype: int64

In [7]:
bills.head()

,state,session,chamber,bill_id,title,created_at,updated_at,type,subjects
0,mt,2011,lower,HB 1,Feed bill,2012-03-08 02:14:49.889000,2012-08-22 02:34:28.186000,bill,"Budget, Spending and Taxes|Legislative Affairs"
1,mt,2011,lower,HB 10,Long-range information technology appropriations,2012-03-08 02:14:50.406000,2012-08-22 02:34:29.128000,bill,"Budget, Spending and Taxes|Technology and Comm..."
2,mt,2011,lower,HB 100,Require fiscal notes to include business impact,2012-03-08 02:14:54.608000,2012-08-22 02:34:37.709000,bill,Legislative Affairs|Trade
3,mt,2011,lower,HB 101,Revise sewer and water district finance laws a...,2012-03-08 02:14:54.691000,2012-08-22 02:34:37.973000,bill,"Budget, Spending and Taxes|Municial and County..."
4,mt,2011,lower,HB 102,Revise probationary driver's license provision...,2012-03-08 02:14:54.741000,2012-08-22 02:34:38.013000,bill,Transportation|Judiciary|Crime


In [6]:
legislators.head()

,leg_id,full_name,first_name,middle_name,last_name,suffixes,nickname,active,state,chamber,district,party,photo_url,created_at,updated_at
0,MTL000001,Ron Arthun,Ron,NaN,Arthun,NaN,NaN,False,mt,NaN,NaN,NaN,http://leg.mt.gov/images/legislators_63rd/1254...,2012-03-08 02:14:25.998000,2015-01-05 17:32:32.204000
1,MTL000002,Shannon Augare,Shannon,NaN,Augare,NaN,NaN,False,mt,NaN,NaN,NaN,http://leg.mt.gov/images/legislators_63rd/1430...,2012-03-08 02:14:26.137000,2015-01-05 17:32:32.226000
2,MTL000003,Joe Balyeat,Joe,NaN,Balyeat,NaN,NaN,False,mt,NaN,NaN,NaN,http://leg.mt.gov/images/legislators_62nd/3704...,2012-03-08 02:14:26.145000,2012-06-19 02:12:08.309000
3,MTL000004,Debby Barrett,Debby,NaN,Barrett,NaN,NaN,False,mt,NaN,NaN,NaN,http://leg.mt.gov/images/legislators_64th/1354...,2012-03-08 02:14:26.154000,2017-01-10 07:15:06.308000
4,MTL000005,Anders Blewett,Anders,NaN,Blewett,NaN,NaN,False,mt,NaN,NaN,NaN,http://leg.mt.gov/images/legislators_63rd/1362...,2012-03-08 02:14:26.161000,2015-01-05 17:32:32.175000


In [8]:
last_actions_17 = actions_17.groupby('bill_id').agg({'action': 'last'})
last_actions_17 = last_actions_17.merge(sponsors_17[['bill_id', 'name', 'leg_id']], how='left', left_index=True, right_on='bill_id')
last_actions_17.rename(columns={'name': 'sponsor'}, inplace=True)
last_actions_17 = last_actions_17.merge(bills[bills['session'] == 2017][['bill_id','title','type']], how='left', left_on='bill_id', right_on='bill_id')

In [9]:
# 2017 bill outcomes
last_actions_17['action'].value_counts()

Died in Standing Committee                  488
Chapter Number Assigned                     440
Died in Process                             100
Filed with Secretary of State                98
Vetoed by Governor                           46
Veto Override Vote Mail Poll in Progress     10
Transmitted to Governor                       3
Printed - Enrolled Version Available          2
Name: action, dtype: int64

In [10]:
def get_bills_by_leg_name(name):
    # assumes last_actions_17 is populated
    return last_actions_17.loc[last_actions_17['sponsor'] == name]

In [11]:
# display bills by outcome
last_actions_17[last_actions_17['action'] == 'Vetoed by Governor']

,action,bill_id,sponsor,leg_id,title,type
107,Vetoed by Governor,HB 198,Kirk Wagoner,MTL000220,Revise distribution of death certificate fee i...,bill
112,Vetoed by Governor,HB 202,Kirk Wagoner,MTL000220,Generally revise laws related to government em...,bill
115,Vetoed by Governor,HB 205,Alan Redfield,MTL000214,Revise tax laws related to vehicles,bill
144,Vetoed by Governor,HB 231,Peggy Webb,MTL000400,Revising laws regarding trespass,bill
160,Vetoed by Governor,HB 246,Randy Brodehl,MTL000063,Revise handgun laws pertaining to federal prop...,bill
170,Vetoed by Governor,HB 255,Brad Tschida,MTL000256,Revise laws related to the governor's use of s...,bill
178,Vetoed by Governor,HB 262,Bill Harris,MTL000372,Generally revise concealed carry regulations f...,bill
182,Vetoed by Governor,HB 266,Nancy Ballance,MTL000173,Authorize interstate healthcare compact,bill
198,Vetoed by Governor,HB 280,Randy Brodehl,MTL000063,Allow Legislators to conceal carry handguns on...,bill
227,Vetoed by Governor,HB 306,Bill Harris,MTL000372,Revise the Montana Federal Mandates Act,bill


In [12]:
# map bill outcomes to sponsors
by_leg = last_actions_17.groupby(['sponsor', 'leg_id', 'action']).agg({'bill_id': 'count'}).reset_index()
by_leg.rename(columns={'action': 'last_action', 'bill_id': 'count'}, inplace=True)
sponsor_record = by_leg.pivot(
    index='leg_id',
    columns='last_action',
    values='count'
).fillna(0.0)
sponsor_record['total'] = sponsor_record.sum(axis=1)
inc_cols = ['leg_id','full_name','chamber','district','party']
sponsor_record = sponsor_record.merge(legislators[legislators['active']][inc_cols], left_index=True, right_on='leg_id', how='left')
sponsor_record = sponsor_record.set_index('full_name')

In [13]:
sponsor_record.sort_values('total', ascending=False)

,Chapter Number Assigned,Died in Process,Died in Standing Committee,Filed with Secretary of State,Printed - Enrolled Version Available,Transmitted to Governor,Veto Override Vote Mail Poll in Progress,Vetoed by Governor,total,leg_id,chamber,district,party
full_name,,,,,,,,,,,,,
Edward Buttrey,11.0,1.0,5.0,10.0,0.0,1.0,1.0,0.0,29.0,MTL000009,upper,11,Republican
Roger Webb,3.0,2.0,12.0,2.0,0.0,0.0,0.0,6.0,25.0,MTL000172,upper,23,Republican
Duane Ankney,9.0,0.0,5.0,8.0,0.0,0.0,0.0,1.0,23.0,MTL000051,upper,20,Republican
Greg Hertz,4.0,4.0,14.0,0.0,0.0,0.0,0.0,0.0,22.0,MTL000190,lower,12,Republican
Keith Regier,5.0,0.0,4.0,8.0,0.0,0.0,1.0,3.0,21.0,MTL000402,upper,3,Republican
Chas Vincent,9.0,1.0,6.0,4.0,0.0,0.0,0.0,0.0,20.0,MTL000043,upper,1,Republican
Jill Cohenour,8.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,20.0,MTL000305,upper,42,Democratic
Jeff Essmann,6.0,2.0,9.0,2.0,0.0,0.0,0.0,0.0,19.0,MTL000012,lower,54,Republican
Kimberly Dudik,11.0,1.0,7.0,0.0,0.0,0.0,0.0,0.0,19.0,MTL000181,lower,94,Democratic


In [14]:
get_bills_by_leg_name("Sue Malek")

,action,bill_id,sponsor,leg_id,title,type
725,Died in Standing Committee,SB 112,Sue Malek,MTL000103,Revise laws relating to corporate loss carryba...,bill
757,Died in Standing Committee,SB 141,Sue Malek,MTL000103,Establish commission to study legislative comp...,bill
764,Died in Standing Committee,SB 148,Sue Malek,MTL000103,Establish the Montana Pay Equity Act,bill
786,Chapter Number Assigned,SB 168,Sue Malek,MTL000103,Revising energy procurement plan review,bill
834,Died in Standing Committee,SB 210,Sue Malek,MTL000103,Revise public service commission laws and qual...,bill
844,Chapter Number Assigned,SB 22,Sue Malek,MTL000103,Revise laws re termination of parental rights ...,bill
849,Chapter Number Assigned,SB 224,Sue Malek,MTL000103,State cabin site exemption for replacement wat...,bill
869,Chapter Number Assigned,SB 242,Sue Malek,MTL000103,Encourage high schools to administer civics test,bill
888,Chapter Number Assigned,SB 26,Sue Malek,MTL000103,Revise laws regarding sexual intercourse witho...,bill
911,Died in Standing Committee,SB 280,Sue Malek,MTL000103,Generally revise DUI laws,bill
